### Preparando o Ambiente

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import re

### Upload Calendario

In [485]:
soup = BeautifulSoup(open('Infos/Complete.html',encoding='utf-8-sig').read())
df_aulas = pd.DataFrame()
for id in [ i for i in range(7)]:
    regex = re.compile(r'>.*?<')
    text = soup.find('div', {"id": "win0divDERIVED_REGFRM1_DESCR20${}".format(id)}).find('td', {'class': 'PAGROUPDIVIDER PSLEFTCORNER'}).prettify().replace('\n','')
    materia = regex.findall(text)[0].replace('> ','').replace('<','')

    df = pd.read_html(soup.find('div', {"id": f"win0divCLASS_MTG_VW${id}"}).prettify())[0]
    df.columns = ['NR_AULA', 'SECAO', 'COMPONENTE', 'HORARIO', 'SALA', 'PROFESSOR', 'DATAS']
    df['START'] = pd.Series(row[1]['DATAS'].split(' - ')[0] for row in df.iterrows())
    df['END'] = pd.Series(row[1]['DATAS'].split(' - ')[1] for row in df.iterrows())
    try:
        df['HR_START'] = pd.Series(row[1]['HORARIO'].split(' - ')[0][4:] for row in df.iterrows())
        df['HR_END'] = pd.Series(row[1]['HORARIO'].split(' - ')[1] for row in df.iterrows())
    except:
        df['HR_START'] = '11:00'
        df['HR_END'] = '12:00'
    df['MATERIA'] = materia
    df.START = pd.to_datetime(df.START + ' ' + df.HR_START, format='%d/%m/%Y %H:%M').dt.strftime('%Y-%m-%dT%H:%M')
    df.END = pd.to_datetime(df.END + ' ' + df.HR_END, format='%d/%m/%Y %H:%M').dt.strftime('%Y-%m-%dT%H:%M')
    df.NR_AULA = df.loc[0].NR_AULA.astype(int)
    df.SECAO = df.loc[0].SECAO
    df.COMPONENTE = df.loc[0].COMPONENTE
    df['SORT_PROF'] = pd.Series(' '.join(row[1]['PROFESSOR'].split(' ')[:2]) for row in df.iterrows())
    df_aulas = pd.concat([df_aulas, df])
df_aulas = df_aulas.reset_index(drop=True)

### Cria String do Array de Eventos Objetos JS

In [487]:
string_json = 'const eventos = ['
cont_aulas = 0
for i in [ row[1] for row in df_aulas.iterrows()]:

    prof = i.SORT_PROF
    professor = i.PROFESSOR
    aula = str(i.NR_AULA)
    secao = i.SECAO
    componente = i.COMPONENTE
    start = i.START
    end = i.END
    materia = i.MATERIA

    string_json += """
        {
          body: {
                  contentType: "text",
                  content:`
                  Professor: """+ professor +"""
                  Nº Aula: """ + aula + """
                  ID Seção: """ + secao + """
                  Modalidade: """ + componente + """
                  `
                },
         start: {
                   dateTime:'"""+ start +"""',
                   timeZone: "E. South America Standard Time"
                },
           end: {
                  dateTime:'"""+ end +"""',
                  timeZone: "E. South America Standard Time"
                },
       subject: '""" + materia + """: """ + prof + """'
        },
    """
    cont_aulas = cont_aulas + 1
string_json +=' ]'

f = open(f"../eventos_aulas.js", "w", encoding='utf-8-sig')
f.write(string_json)
f.close()